# Momentum ETF Performance

In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import DataReader as pdr
import yfinance as yf
import plotly.graph_objects as go
import statsmodels.api as sm
pd.options.display.float_format = '{:.4f}'.format

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Function to pull returns
def returns(tickers):
    ret = yf.download(tickers, start='2000-01-01', end='2024-12-31', progress=False)
    ret = ret["Close"].resample("ME").last()
    ret = ret.pct_change()
    ret.columns = tickers
    ret.index = ret.index.to_period('M')
    return ret

In [3]:
# Pull data
ticker_list = ['MTUM','PDP','XMMO','ONEO','VFMO','JMOM','SPMO','QMOM','FDMO','MMTM']

# Pull the data from Yahoo
df = returns(ticker_list)
df

YF.download() has changed argument auto_adjust default to True


,MTUM,PDP,XMMO,ONEO,VFMO,JMOM,SPMO,QMOM,FDMO,MMTM
Date,,,,,,,,,,
2005-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.0368
2005-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0678
2005-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0432
2005-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0343
...,...,...,...,...,...,...,...,...,...,...
2024-08,0.0254,0.0288,0.0239,0.0335,0.0145,0.0258,0.0293,0.0378,0.0150,-0.0116
2024-09,0.0244,0.0269,0.0207,0.0295,0.0204,0.0216,0.0382,0.0163,0.0183,0.0176
2024-10,0.0009,-0.0007,-0.0001,-0.0017,-0.0075,0.0033,-0.0065,0.0020,0.0060,0.0079


In [4]:
# Add factors
ff3 = pdr('F-F_Research_Data_Factors','famafrench', start=1900)[0]/100
Mom = pdr('F-F_Momentum_Factor','famafrench', start=1900)[0]/100
Mom.columns = ['Mom']
ff = ff3.join(Mom)
df = df.join(ff)

## Run market model and Fama-French-Carhart performance models

In [5]:
stats = pd.DataFrame(dtype=float, columns=pd.MultiIndex.from_product([['CAPM','FF4'],['alpha','beta','t_alpha','t_beta','nobs']]), index=ticker_list)
stats

CAPM                            FF4                         
     alpha beta t_alpha t_beta nobs alpha beta t_alpha t_beta nobs
MTUM   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
PDP    NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
XMMO   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
ONEO   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
VFMO   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
JMOM   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
SPMO   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
QMOM   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
FDMO   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN
MMTM   NaN  NaN     NaN    NaN  NaN   NaN  NaN     NaN    NaN  NaN

In [6]:
# Run the performance evaluations
for tick in ticker_list:
    # Market-model
    mm = sm.OLS(df[tick]-df['RF'], sm.add_constant(df['Mkt-RF']),missing='drop').fit()
    stats.loc[tick, ('CAPM','alpha')] = mm.params[0]
    stats.loc[tick, ('CAPM','beta')]  = mm.params[1]
    stats.loc[tick, ('CAPM','t_alpha')] = mm.tvalues[0]
    stats.loc[tick, ('CAPM','t_beta')]  = mm.tvalues[1]
    stats.loc[tick, ('CAPM','nobs')]  = mm.nobs

    # Fama-French 4-factor model
    ff4 = sm.OLS(df[tick]-df['RF'], sm.add_constant(df[['Mkt-RF','SMB','HML','Mom']]),missing='drop').fit()
    stats.loc[tick, ('FF4','alpha')] = ff4.params[0]
    stats.loc[tick, ('FF4','beta')]  = ff4.params[1]
    stats.loc[tick, ('FF4','coeff_smb')]  = ff4.params[2]
    stats.loc[tick, ('FF4','coeff_hml')]  = ff4.params[3]
    stats.loc[tick, ('FF4','coeff_mom')]  = ff4.params[4]
    stats.loc[tick, ('FF4','t_alpha')] = ff4.tvalues[0]
    stats.loc[tick, ('FF4','t_beta')]  = ff4.tvalues[1]  
    stats.loc[tick, ('FF4','nobs')]    = ff4.nobs 
stats

CAPM                                     FF4                         \
       alpha   beta t_alpha  t_beta     nobs   alpha   beta t_alpha  t_beta   
MTUM  0.0003 0.9457  0.2385 33.1512  99.0000 -0.0012 1.0346 -1.3099 49.0751   
PDP   0.0006 0.9846  0.4241 37.0406  85.0000 -0.0005 1.0456 -0.5439 49.3663   
XMMO  0.0009 0.9260  0.9014 41.4577 146.0000 -0.0003 0.9979 -0.3613 48.6811   
ONEO  0.0013 0.9050  0.7382 22.9930 140.0000 -0.0012 1.0463 -0.9490 35.3316   
VFMO -0.0033 1.0494 -2.1512 32.8676 108.0000 -0.0026 1.0272 -2.2665 38.0721   
JMOM -0.0011 1.0353 -0.7678 34.8529 213.0000 -0.0021 1.1030 -1.7694 39.8485   
SPMO -0.0019 1.1372 -0.5235 14.6403 108.0000 -0.0022 1.2206 -0.7714 18.7520   
QMOM  0.0033 0.8730  1.5179 18.8650 110.0000  0.0011 1.0398  0.7118 27.3942   
FDMO -0.0005 1.0569 -0.1882 21.3917  82.0000 -0.0000 1.0709 -0.0110 34.6366   
MMTM  0.0010 1.0554  0.6499 30.6012 237.0000  0.0006 1.0699  0.4639 31.6463   

                                             
         nobs coeff_smb coeff_hml coeff_mom  
MTUM  99.0000   -0.0145   -0.0784    0.2454  
PDP   85.0000    0.0402   -0.0859    0.1978  
XMMO 146.0000   -0.1367    0.0053    0.1439  
ONEO 140.0000   -0.0723   -0.0621    0.3902  
VFMO 108.0000    0.1545    0.2452    0.0499  
JMOM 213.0000    0.1537   -0.1465    0.2085  
SPMO 108.0000    0.7181    0.0780    0.6700  
QMOM 110.0000   -0.1469    0.0028    0.3952  
FDMO  82.0000    0.5903    0.1543    0.3939  
MMTM 237.0000    0.3517   -0.0590    0.2112

In [7]:
# Sort to find the highest historical alpha
stats.sort_values(by=[('CAPM','t_alpha')], ascending=False)

CAPM                                     FF4                         \
       alpha   beta t_alpha  t_beta     nobs   alpha   beta t_alpha  t_beta   
QMOM  0.0033 0.8730  1.5179 18.8650 110.0000  0.0011 1.0398  0.7118 27.3942   
XMMO  0.0009 0.9260  0.9014 41.4577 146.0000 -0.0003 0.9979 -0.3613 48.6811   
ONEO  0.0013 0.9050  0.7382 22.9930 140.0000 -0.0012 1.0463 -0.9490 35.3316   
MMTM  0.0010 1.0554  0.6499 30.6012 237.0000  0.0006 1.0699  0.4639 31.6463   
PDP   0.0006 0.9846  0.4241 37.0406  85.0000 -0.0005 1.0456 -0.5439 49.3663   
MTUM  0.0003 0.9457  0.2385 33.1512  99.0000 -0.0012 1.0346 -1.3099 49.0751   
FDMO -0.0005 1.0569 -0.1882 21.3917  82.0000 -0.0000 1.0709 -0.0110 34.6366   
SPMO -0.0019 1.1372 -0.5235 14.6403 108.0000 -0.0022 1.2206 -0.7714 18.7520   
JMOM -0.0011 1.0353 -0.7678 34.8529 213.0000 -0.0021 1.1030 -1.7694 39.8485   
VFMO -0.0033 1.0494 -2.1512 32.8676 108.0000 -0.0026 1.0272 -2.2665 38.0721   

                                             
         nobs coeff_smb coeff_hml coeff_mom  
QMOM 110.0000   -0.1469    0.0028    0.3952  
XMMO 146.0000   -0.1367    0.0053    0.1439  
ONEO 140.0000   -0.0723   -0.0621    0.3902  
MMTM 237.0000    0.3517   -0.0590    0.2112  
PDP   85.0000    0.0402   -0.0859    0.1978  
MTUM  99.0000   -0.0145   -0.0784    0.2454  
FDMO  82.0000    0.5903    0.1543    0.3939  
SPMO 108.0000    0.7181    0.0780    0.6700  
JMOM 213.0000    0.1537   -0.1465    0.2085  
VFMO 108.0000    0.1545    0.2452    0.0499

In [8]:
alpha = stats.loc['SPMO',('CAPM','alpha')]*12
print(f'{alpha: .2%}')

-2.33%


In [9]:
ff4 = sm.OLS(df['SPMO']-df['RF'], sm.add_constant(df[['Mkt-RF','SMB','HML','Mom']]),missing='drop').fit()
print(ff4.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.822
Model:                            OLS   Adj. R-squared:                  0.815
Method:                 Least Squares   F-statistic:                     118.6
Date:                Wed, 16 Apr 2025   Prob (F-statistic):           1.22e-37
Time:                        20:25:42   Log-Likelihood:                 235.57
No. Observations:                 108   AIC:                            -461.1
Df Residuals:                     103   BIC:                            -447.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0022      0.003     -0.771      0.4